In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from datetime import datetime
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler

from sklearn.pipeline import make_pipeline
from sklearn.pipeline import make_union
from sklearn.compose import make_column_transformer

import sys
sys.path.append("/home/kieran/code/Ant-mel/legendary_game_recs/")

from preprocessing.preprocess_1_cleaning import *
from preprocessing.preprocess_2_features import *
from preprocessing.pipeline_the_unification import *

In [4]:
raw_data = pd.read_csv("../raw_data/all_game_data_v1_corrected2.csv", low_memory=False)
topic_data = pd.read_csv("../raw_data/topics.csv", low_memory=False)

In [6]:
reference_data = pipeline_genre_ohe_only(raw_data)

/home/kieran/code/Ant-mel/legendary_game_recs/preprocessing/preprocess_1_cleaning.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column]=pd.to_datetime(df[column])
/home/kieran/code/Ant-mel/legendary_game_recs/preprocessing/preprocess_2_features.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dupe['gen'] = None
/home/kieran/code/Ant-mel/legendary_game_recs/preprocessing/preprocess_1_cleaning.py:161: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

In [7]:
topic_data

,0,1,2,3,4,5,6,7,8,9
0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,1,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
36815,0,0,0,0,1,0,0,0,0,0
36816,0,0,0,0,0,0,0,1,0,0
36817,0,0,0,1,0,0,0,0,0,0
36818,0,0,0,0,0,0,1,0,0,0


In [8]:
reference_data = pd.concat((reference_data,topic_data), axis=1)

In [10]:
reference_data.columns

Index(['title', 'release_date', 'plays', 'playing', 'backlogs', 'wishlist',
       'developers', 'avg_review', 'platforms', 'description', 'total_reviews',
       'total_lists', 'main', 'ratings_zero_five', 'ratings_one_zero',
       'ratings_one_five', 'ratings_two_zero', 'ratings_two_five',
       'ratings_three_zero', 'ratings_three_five', 'ratings_four_zero',
       'ratings_four_five', 'ratings_five_zero', 'image', 'url', 'Adventure',
       'Indie', 'RPG', 'Simulator', 'Puzzle', 'Shooter', 'Strategy',
       'Platform', 'Arcade', 'Sport', 'gen_1', 'gen_2', 'gen_3', '0', '1', '2',
       '3', '4', '5', '6', '7', '8', '9'],
      dtype='object')

In [11]:
processed_data = drop_unnecesary_coulumns(reference_data)

In [12]:
processed_data.head()

,plays,playing,backlogs,wishlist,avg_review,total_reviews,total_lists,Adventure,Indie,RPG,...,0,1,2,3,4,5,6,7,8,9
0,7,0,2,2,2.2,1,4,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,80,1,82,297,3.3,8,104,0,1,0,...,0,0,0,0,0,0,0,1,0,0
2,9,0,5,1,2.4,0,3,1,0,1,...,0,0,0,0,0,1,0,0,0,0
3,10,3,9,41,4.5,5,12,1,0,0,...,0,0,1,0,0,0,0,0,0,0
4,16,0,26,155,2.1,12,27,1,1,0,...,0,0,0,0,1,0,0,0,0,0


In [13]:
X_train = processed_data.drop(['avg_review'], axis=1)
y_train = processed_data[['avg_review']]

In [14]:
X_train.head()

,plays,playing,backlogs,wishlist,total_reviews,total_lists,Adventure,Indie,RPG,Simulator,...,0,1,2,3,4,5,6,7,8,9
0,7,0,2,2,1,4,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,80,1,82,297,8,104,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,9,0,5,1,0,3,1,0,1,1,...,0,0,0,0,0,1,0,0,0,0
3,10,3,9,41,5,12,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,16,0,26,155,12,27,1,1,0,0,...,0,0,0,0,1,0,0,0,0,0


In [15]:
y_train

,avg_review
0,2.2
1,3.3
2,2.4
3,4.5
4,2.1
...,...
36815,3.8
36816,2.7
36817,2.2
36818,2.0


In [16]:
num_transformer = Pipeline([('imputer', SimpleImputer(strategy="median")),
                             ('mm_scaler', MinMaxScaler())
                            ])

In [17]:
X_train_scaled = num_transformer.fit_transform(X_train)

In [18]:
pd.DataFrame(X_train_scaled)

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,0.000130,0.000161,0.000167,0.000313,0.000263,0.000690,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.001717,0.000323,0.006833,0.046406,0.002105,0.017931,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.000174,0.000161,0.000417,0.000156,0.000000,0.000517,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.000196,0.000645,0.000750,0.006406,0.001316,0.002069,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000326,0.000161,0.002167,0.024219,0.003158,0.004655,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36815,0.000065,0.000161,0.000083,0.000625,0.000000,0.000345,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
36816,0.000109,0.000161,0.000000,0.000156,0.000263,0.000345,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
36817,0.000043,0.000161,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
36818,0.000370,0.000161,0.000000,0.000000,0.000526,0.001207,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


# Model and Finetuning

In [19]:
# Import the necessary libraries
from sklearn.inspection import permutation_importance
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier  # Import KNeighborsClassifier or KNeighborsRegressor as needed

# Fit a KNN model to your data
knn_model = KNeighborsRegressor(n_neighbors=11).fit(X_train_scaled, y_train)  # You can adjust the number of neighbors (k) as needed
# X and y are your feature matrix and target variable



In [ ]:
# Perform permutation importance analysis
permutation_score = permutation_importance(knn_model, X_train_scaled, y_train, n_repeats=10)



In [ ]:
# Create a DataFrame to store the results
importance_df = pd.DataFrame(np.vstack((X_train.columns, permutation_score.importances_mean)).T)
importance_df.columns = ['feature', 'score_decrease']

# Sort the DataFrame by feature importance in descending order
importance_df = importance_df.sort_values(by='score_decrease', ascending=False)

In [ ]:
importance_df

# Prediction

In [20]:
reference_data[reference_data['title'] == "The Legend of Zelda: Breath of the Wild"]

,title,release_date,plays,playing,backlogs,wishlist,developers,avg_review,platforms,description,...,0,1,2,3,4,5,6,7,8,9
11702,The Legend of Zelda: Breath of the Wild,2017-03-03,42000,3500,6600,3400,"[Nintendo EPD Production Group No. 3, Nintendo]",4.4,"[Wii U, Nintendo Switch]",The Legend of Zelda: Breath of the Wild is the...,...,0,0,0,0,0,0,0,1,0,0


In [21]:
reference_data[reference_data['title'] == "Call of Duty: Modern Warfare 3"]

,title,release_date,plays,playing,backlogs,wishlist,developers,avg_review,platforms,description,...,0,1,2,3,4,5,6,7,8,9
19339,Call of Duty: Modern Warfare 3,2011-11-08,14000,43,724,293,"[Sledgehammer Games, Activision]",3.3,"[Windows PC, Mac, Xbox 360, PlayStation 3, Wii]",Call of Duty: Modern Warfare 3 is a first-pers...,...,1,0,0,0,0,0,0,0,0,0


In [22]:
reference_data[reference_data['title'] == "Wipeout"]

,title,release_date,plays,playing,backlogs,wishlist,developers,avg_review,platforms,description,...,0,1,2,3,4,5,6,7,8,9
32334,Wipeout,1995-09-29,368,4,123,71,[Psygnosis],3.2,"[PlayStation 3, Sega Saturn, PlayStation, PC DOS]","Wipeout is a racing game that is set in 2052, ...",...,0,0,0,1,0,0,0,0,0,0


In [42]:
reference_data[reference_data['title'] == "Donkey Kong Country"]

,title,release_date,plays,playing,backlogs,wishlist,developers,avg_review,platforms,description,...,0,1,2,3,4,5,6,7,8,9
32871,Donkey Kong Country,1994-11-21,15000,271,2600,666,"[Hyundai, Rare]",3.9,"[Wii U, Super Famicom, SNES, Wii, New Nintendo...",Donkey Kong Country is a side scrolling platfo...,...,0,1,0,0,0,0,0,0,0,0


In [23]:
zelda_array = X_train_scaled[11702:11703]

In [24]:
omori_array = X_train_scaled[4532:4534]

In [25]:
border_array = X_train_scaled[7157:7158]

In [26]:
wipeout_array = X_train_scaled[32334:32335]

In [43]:
dk_array = X_train_scaled[32871:32872]

In [27]:
 array_inputs = np.concatenate((border_array, zelda_array))

In [44]:
ind_list = list(knn_model.kneighbors(dk_array,n_neighbors=5000)[1][0])

In [45]:
output_df = reference_data.iloc[ind_list, :]

In [46]:
output_df.head(15)

,title,release_date,plays,playing,backlogs,wishlist,developers,avg_review,platforms,description,...,0,1,2,3,4,5,6,7,8,9
32871,Donkey Kong Country,1994-11-21,15000,271,2600,666,"[Hyundai, Rare]",3.9,"[Wii U, Super Famicom, SNES, Wii, New Nintendo...",Donkey Kong Country is a side scrolling platfo...,...,0,1,0,0,0,0,0,0,0,0
36110,Super Mario Bros.,1985-09-13,23000,96,1000,299,"[Playtronic, Nintendo R&D4]",3.5,"[Wii U, Arcade, Wii, Family Computer Disk Syst...",A side scrolling 2D platformer and first entry...,...,0,1,0,0,0,0,0,0,0,0
34882,Super Mario World,1990-11-21,23000,285,2400,599,"[Hyundai, Nintendo EAD]",4.2,"[Wii U, Super Famicom, Arcade, SNES, Wii, New ...",A 2D platformer and first entry on the SNES in...,...,0,1,0,0,0,0,0,0,0,0
33466,Castlevania: Rondo of Blood,1993-10-29,3700,85,1400,612,[Konami],4.1,"[Wii, Turbografx-16/PC Engine CD, PlayStation ...","In Castlevania X: Rondo of Blood, the player t...",...,0,1,0,0,0,0,0,0,0,0
34510,Castlevania II: Belmont's Revenge,1991-07-12,822,7,287,90,[Konami],2.8,[Game Boy],Fifteen years after vanquishing the emperor of...,...,0,1,0,0,0,0,0,0,0,0
33871,Taz-Mania,1992-12-25,351,1,33,20,"[Tec Toy, Sega]",2.1,"[Sega Mega Drive/Genesis, Sega Master System, ...",A video game adaptation of the Taz-Mania carto...,...,0,1,0,0,0,0,0,0,0,0
33983,Felix the Cat,1992-10-01,322,3,40,32,[Hudson Soft],3.4,[NES],The game involves the player controlling Felix...,...,0,1,0,0,0,0,0,0,0,0
33069,The Great Circus Mystery Starring Mickey & Minnie,1994-06-01,275,1,56,22,"[Playtronic, Capcom]",3.2,"[Sega Mega Drive/Genesis, SNES]",The game ditches the alternating two-player me...,...,0,1,0,0,0,0,0,0,0,0
32760,Pitfall: The Mayan Adventure,1994-12-31,297,1,60,26,"[Playtronic, Redline Games]",2.8,"[Windows PC, Sega Mega Drive/Genesis, SNES, Se...",Pitfall: The Mayan Adventure is a side-scrolli...,...,0,1,0,0,0,0,0,0,0,0
33241,Pop'n Twinbee: Rainbow Bell Adventures,1994-01-07,86,1,65,32,[Konami],3.6,"[Wii U, Super Famicom, SNES]","You play as either Twinbee, Gwinbee or Winbee....",...,0,1,0,0,0,0,0,0,0,0


In [ ]:
year_limit = '1997'

In [ ]:
rslt_df = output_df[output_df['release_date'] <= '1997-01-01']

In [ ]:
rslt_df.head(20)